In [1]:
from src.config.data_loader_config import DATA_LOADER_CONFIG, OPTIMIZER_CONFIG
from src.data_loader.loader import Dataloader
from src.model.model import Model, Models, LossFunctions
from src.trainer.predict import save_result
from utils.fix_seed import set_seed
import src.callback as callback
import pytorch_lightning as pl
import torch

# Parameters 설정
batch_size = DATA_LOADER_CONFIG['batch_size']
shuffle = DATA_LOADER_CONFIG['shuffle']
learning_rate = 1e-5
max_epoch = OPTIMIZER_CONFIG['max_epoch']
num_workers: int = DATA_LOADER_CONFIG.get('num_workers', 4)  # num_workers 기본값 4로 설정
torch.cuda.empty_cache()
set_seed(0)

c:\Users\승범 pc\Desktop\study\deeplearning\naver_boot\level1-semantictextsimilarity-nlp-07\project1_virtual\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Seed set to 0


In [2]:
# 모델 선언
# select Model 
model_name = 'electra_base_v3'
model = Model(Models.electra_base_v3, learning_rate, LossFunctions.mse_loss)

# Dataloader 선언
dataloader = Dataloader(batch_size, shuffle, Models.electra_base_v3, check_aug=False, num_workers=num_workers)

# callback 정의
epoch_print_callback = callback.EpochPrintCallback()
checkpoint_callback = callback.ModelCheckpoint(model_name=model_name)
early_stopping = callback.EarlyStopping()
lr_monitor = callback.LearningRateMonitor()

trainer = pl.Trainer(
    accelerator='auto',
    devices='auto',
    max_epochs=max_epoch,
    callbacks=[lr_monitor, epoch_print_callback, checkpoint_callback, early_stopping],
    precision='16-mixed',
    deterministic=True
)

c:\Users\승범 pc\Desktop\study\deeplearning\naver_boot\level1-semantictextsimilarity-nlp-07\project1_virtual\Lib\site-packages\pytorch_lightning\utilities\parsing.py:208: Attribute 'loss_func' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss_func'])`.
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


# 학습

In [ ]:
trainer.fit(model=model, datamodule=dataloader)

# 추론


In [3]:
best_model_path = checkpoint_callback.best_model_path
model = Model.load_from_checkpoint(best_model_path)
trainer.test(model=model, datamodule=dataloader)


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You are using a CUDA device ('NVIDIA GeForce RTX 4070 Ti SUPER') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
c:\Users\승범 pc\Desktop\study\deeplearning\naver_boot\level1-semantictextsimilarity-nlp-07\project1_virtual\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will 

Testing DataLoader 0: 100%|██████████| 35/35 [00:01<00:00, 21.11it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_pearson          0.9232333302497864
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_pearson': 0.9232333302497864}]

In [4]:
# 추론
predictions = trainer.predict(model=model, datamodule=dataloader)
test_predictions, val_predictions = predictions[0], predictions[1]
# 결과 저장
save_result(test_predictions, model_name, max_epoch, mode='output')
save_result(val_predictions, model_name, max_epoch, mode='train')

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting DataLoader 1: 100%|██████████| 35/35 [00:01<00:00, 23.16it/s]
